## significance

In [ ]:
comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for mut_comps_l in muts_df["imodulons"]:

    if len(mut_comps_l) > 0:
        for mut_comp_d in mut_comps_l:
            if mut_comp_d["name"] in comp_mut_df.index:
                comp_mut_df.loc[mut_comp_d["name"],
                                "observed mutation count"] += 1
            else:
                comp_mut_df.loc[mut_comp_d["name"],
                                "observed mutation count"] = 1
    else:
        if "unknown" in comp_mut_df.index:
            comp_mut_df.loc["unknown", "observed mutation count"] += 1
        else:
            comp_mut_df.loc["unknown", "observed mutation count"] = 1

display(comp_mut_df.head())

In [ ]:
for _, r in muts_df.iterrows():
    for feat_d in r["imodulons"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]
                                                       ]["observed mutation count"]

In [ ]:
imod_len_df = pd.read_pickle("./data/imodulon_len_df.pkl")
imod_len_df.head()

In [ ]:
import numpy as np


comp_mut_df["length"] = comp_mut_df.apply(
    lambda r: imod_len_df.loc[r.index, "imodulon length"])
total_seq_len = np.sum(comp_mut_df["length"])
comp_mut_df["proportion"] = comp_mut_df["length"].apply(
    lambda comp_len: comp_len/total_seq_len)
comp_mut_df.head()

In [ ]:
from collections import Counter

comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.

num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=int(num_muts),
        p=proportion_l,
        replace=True,
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(
    lambda p_val: True if p_val < 0.05 else False)

display(comp_mut_df.head())

In [ ]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=0.05,
    method="bonferroni")

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

In [ ]:
# not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(
    lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
comp_mut_df.head()

In [ ]:
for i, r in muts_df.iterrows():
    for d in r["imodulons"]:
        if d["name"] in comp_mut_df.index:
            d["significant"] = comp_mut_df.loc[d["name"]]["final significance"]

In [ ]:
muts_df.to_pickle("./data/9_df.pkl")